# MMVCのリポジトリをGoogle Driveに保存する

ver.2022/10/3

MMVC_Trainerのリポジトリと事前学習済みモデルを、Google Driveに保存します。

​

In [ ]:
#@title ## 0 Notebookの準備
#@markdown このNotebookの実行に必要なパッケージをインストールします。

#@markdown 「警告: このノートブックは Google が作成したものではありません。」といったポップアップが表示された場合、内容を確認して「このまま実行」を選択してください。このノートブックでは、外部へのデータ送信は一切行われません。

!pip install ipywidgets
import ipywidgets as widgets

​

In [ ]:
#@title ## 1 Google Driveをマウント
#@markdown **このノートブックで、Google Driveを使用するための設定です。**

#@markdown 　「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されるので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。

#@markdown 成功すれば、下記メッセージが出ます。 { display-mode: "form" }

#@markdown ``` 
#@markdown Mounted at /content/drive/
#@markdown ```

from google.colab import drive
drive.mount('/content/drive')

​

In [ ]:
#@title ## 2 MMVC_Trainerのクローン
#@markdown セルを実行すると、保存先のパスの入力欄と実行ボタンが表示されます。

#@markdown 必要に応じてパスを変更した後、実行をクリックしてください。

#@markdown パスで指定したディレクトリ上に、MMVC_Trainerというフォルダが作成されます。

#@markdown 成功すると、最後に以下のようなメッセージが表示されます。
#@markdown ```
#@markdown Checking out files: 100% (902/902), done.
#@markdown ```

def clone_repo():
  directory_input = widgets.Text(value='/content/drive/MyDrive', placeholder='/content/drive/MyDrive', description='Directory:', disabled=False)
  clone_repo_excuse = widgets.Button(description='実行', disabled=False)

  def clone_repo_on_click(b: widgets.Button) -> None:
    global directory
    directory = directory_input.value
    %cd $directory
    !git clone --depth=1 https://github.com/isletennos/MMVC_Trainer/

  clone_repo_excuse.on_click(clone_repo_on_click)
  display(directory_input, clone_repo_excuse)

clone_repo()

​

In [ ]:
#@title ## 3 fine_modelのダウンロード
#@markdown 事前学習済みのモデルをダウンロードします。

#@markdown セルを実行し、Colabを使用している場合はColab、それ以外の場合はOtherを選択してから実行をクリックしてください。

def download_finemodel():
  using_colab_input = widgets.Dropdown(options=["Colab", "Other"], value="Colab", description='Using_Colab:', disabled=False)
  download_finemodel_excuse = widgets.Button(description='実行', disabled=False)

  def download_finemodel_on_click(b: widgets.Button) -> None:
      using_colab = using_colab_input.value
      if using_colab == "Colab":
        import gdown
        %cd {directory}/MMVC_Trainer/fine_model/
        gdown.download('https://drive.google.com/uc?id=1MRmB-9EL2-xhxIIraVx20qWqyX-8ZMPw', 'G_180000.pth', quiet=False)
        gdown.download('https://drive.google.com/uc?id=1oPbMM7cGz-z0fLgj7DW_wPGRCwvgD2jZ', 'D_180000.pth', quiet=False)
      else:
        %cd {directory}/MMVC_Trainer/fine_model/
        !fileid="1MRmB-9EL2-xhxIIraVx20qWqyX-8ZMPw"; filename="G_180000.pth"; html=`curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=${fileid}"`; curl -Lb ./cookie "https://drive.google.com/uc?export=download&`echo ${html}|grep -Po '(confirm=[a-zA-Z0-9\-_]+)'`&id=${fileid}" -o ${filename}
        !fileid="1oPbMM7cGz-z0fLgj7DW_wPGRCwvgD2jZ"; filename="D_180000.pth"; html=`curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=${fileid}"`; curl -Lb ./cookie "https://drive.google.com/uc?export=download&`echo ${html}|grep -Po '(confirm=[a-zA-Z0-9\-_]+)'`&id=${fileid}" -o ${filename}

  download_finemodel_excuse.on_click(download_finemodel_on_click)
  display(using_colab_input, download_finemodel_excuse)

download_finemodel()

​

In [ ]:
#@title ## 4 確認
#@markdown 以下のようなメッセージが表示されれば、保存が正常に完了しています。
#@markdown ```
#@markdown Directory: MMVC_Trainer
#@markdown attentions.py
#@markdown commons.py
#@markdown ...(略)
#@markdown
#@markdown Directory: MMVC_Trainer/fine_model
#@markdown D_180000.pth
#@markdown G_180000.pth
#@markdown ```

print("Directory: MMVC_Trainer")
!ls -1 {directory}/MMVC_Trainer
print("\n")
print("Directory: MMVC_Trainer/fine_model")
!ls -1 {directory}/MMVC_Trainer/fine_model